In [2]:
# import transformers
import sys
import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from scipy.misc import imread, imresize
from tqdm import tqdm
# from skimage.transform import resize
# from skimage.io import imread
from PIL import Image
sys.path.insert(0, '../src')
from caption import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

In [5]:
with open('../config/caption_local.json') as f:
    jsonread = json.load(f) 
    model_fp = jsonread['model_fp']
    wordmap_fp=jsonread['wordmap_fp']
    img_fp=jsonread['img_fp']
    beam_size=jsonread['beam_size']
    smooth=jsonread['smooth']

In [6]:
import os
os.chdir('../src/')

In [7]:
# Load model
checkpoint = torch.load(model_fp, map_location=str(device))
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval();

In [8]:
def get_viz(wordmap_fp, img_fp, beam_size, fname, smooth, visualize=True):
    # Load word map (word2ix)
    with open(wordmap_fp, 'r') as j:
        word_map = json.load(j)
    rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

#     Encode, decode with attention and beam search
#     seq, alphas = caption_image_beam_search(encoder, decoder, img_fp, word_map, beam_size)
    seq, alphas = caption_image_beam_search(encoder, decoder, img_fp, word_map, beam_size)
    alphas = torch.FloatTensor(alphas)
    
    words = [rev_word_map[ind] for ind in seq]
    # Visualize caption and attention of best sequence
#     visualize_att(img_fp, seq, alphas, rev_word_map, fname, smooth)
    if visualize == True: visualize_att(img_fp, seq, alphas, rev_word_map, smooth)
    return ' '.join(words[1:-1])

In [15]:
pwd

'/home/jdlevy/Documents/UCSD/Senior/Quarter 2/DSC 180B/180b_capstone_xai/src'

In [9]:
data_dir = '../data/temp/imgs/'
out_dir = '../data/out'
img_caption_dict = {}
for img_folder in tqdm(os.listdir(data_dir)):
    load_fp = data_dir+'/'+img_folder
    for file in tqdm(os.listdir(f'{data_dir}/{img_folder}')):
        in_fp = load_fp+'/'+file
        out_fp = out_dir+'/'+img_folder+'/'
#         print(file)
        if 'raw' in file:
            print(file)
            print(img_folder)
            out_fp+=f'maps_raw_{img_folder}.png'
            img_caption_dict[int(img_folder)] = get_viz(wordmap_fp, in_fp, beam_size, out_fp, smooth, visualize=False)
            
# #             print(out_fp)
# #             get_viz(wordmap_fp, in_fp, beam_size, out_fp, smooth, visualize=False)
#         if 'output' in file:
#             ann_id = file.split('_')[-1].strip('.png')
#             out_fp+=f'maps_{ann_id}.png'
#             words = get_viz(wordmap_fp, in_fp, beam_size, out_fp, smooth, visualize=False)
# #             print(out_fp)
#             ann_caption_dict[ann_id] = ' '.join(words[1:-1])
            

  0%|          | 0/13 [00:00<?, ?it/s]

raw.png
31093



100%|██████████| 3/3 [00:00<00:00, 46.87it/s]

100%|██████████| 17/17 [00:00<00:00, 322.43it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

raw.png
311909
raw.png
466156
raw.png
295316
raw.png
491464


100%|██████████| 5/5 [00:00<00:00, 81.62it/s]

100%|██████████| 9/9 [00:00<00:00, 182.90it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

raw.png
551215
raw.png
548555
raw.png
326542
raw.png
25394


100%|██████████| 10/10 [00:01<00:00,  6.48it/s]

raw.png
455624


In [78]:
data_dir = '../data/out'
out_dir = '../data/out'
ann_caption_dict = {}
for img_folder in tqdm(os.listdir(data_dir)):
#     try: ann_caption_dict[int(img_folder)] = [[]]
#     except ValueError: pass
    load_fp = data_dir+'/'+img_folder
    for file in tqdm(os.listdir(f'{data_dir}/{img_folder}')):
        in_fp = load_fp+'/'+file
        out_fp = out_dir+'/'+img_folder+'/'
#         print(file)
#         if 'raw' in file:
#             print(file)
#             out_fp+='maps_raw_.png'
#             img_caption_dict[img_id]
# #             print(out_fp)
#             get_viz(wordmap_fp, in_fp, beam_size, out_fp, smooth, visualize=False)
        if 'output' in file:
            ann_id = file.split('_')[-1].strip('.png')
            out_fp+=f'maps_{ann_id}.png'
            words = get_viz(wordmap_fp, in_fp, beam_size, out_fp, smooth, visualize=False)
#             print(out_fp)
            ann_caption_dict[(int(img_folder), int(ann_id))] = [words]
            

100%|██████████| 3/3 [00:00<00:00, 54.25it/s]

100%|██████████| 23/23 [00:00<00:00, 310189.68it/s]

100%|██████████| 5/5 [00:00<00:00, 59.33it/s]

100%|██████████| 2/2 [00:00<00:00, 32.24it/s]

100%|██████████| 11/11 [00:02<00:00,  4.81it/s]


In [64]:
pd.Series(img_caption_dict).index.apply(lambda x: ann_caption_dict[x])

AttributeError: 'Int64Index' object has no attribute 'apply'

In [73]:
pd.DataFrame(ann_caption_dict)

ValueError: If using all scalar values, you must pass an index

In [94]:
ann_df = pd.DataFrame(ann_caption_dict).transpose().reset_index().rename(columns={0:'ann_caption'})

In [86]:
img_caption_dict

{31093: 'a man riding a skateboard up the side of a ramp',
 311909: 'a blue and white bus parked in a parking lot',
 466156: 'a cat sitting on the hood of a car',
 295316: 'a man riding a wave on top of a surfboard',
 491464: 'a young boy holding a baseball bat on a field',
 551215: 'a man holding a tennis racquet on a tennis court',
 548555: 'a hot dog with ketchup and mustard on a bun',
 326542: 'a man riding skis down a snow covered slope',
 25394: 'a man sitting at a table with a glass of wine',
 455624: 'a man riding on the back of a motorcycle'}

In [96]:
caption_df = pd.Series(img_caption_dict).to_frame().rename(columns={0:'raw_caption'}).merge(ann_df, left_index=True, right_on='level_0')

In [100]:
caption_df.head()

,raw_caption,level_0,level_1,ann_caption
0,a man riding a skateboard up the side of a ramp,31093,2157893,a man riding a skateboard up the side of a ramp
1,a man riding a skateboard up the side of a ramp,31093,1233457,a man riding a skateboard up the side of a ramp
2,a man riding a skateboard up the side of a ramp,31093,453234,a man riding a skateboard up the side of a ramp
3,a man riding a skateboard up the side of a ramp,31093,638282,a man riding a skateboard up the side of a ramp
4,a man riding a skateboard up the side of a ramp,31093,1421746,a man riding a skateboard up the side of a ramp


### BERT

In [103]:
sentences = [
    "This is an awesome book to learn NLP.",
    "DistilBERT is an amazing NLP model.",
    "We can interchangeably use embedding, encoding, or vectorizing.",
]

vectorizer = Vectorizer()
vectorizer.bert(sentences)
vectors_bert = vectorizer.vectors


In [104]:
dist_1 = spatial.distance.cosine(vectors_bert[0], vectors_bert[1])
dist_2 = spatial.distance.cosine(vectors_bert[0], vectors_bert[2])
print('dist_1: {0}, dist_2: {1}'.format(dist_1, dist_2))

dist_1: 0.0, dist_2: 0.042955875396728516


In [9]:
import json

In [13]:
with open('../config/gen_inpaint.json') as f:
    params = json.load(f)
    data_dir = params['data_dir']
    input_dir = params['input_dir']    
    temp_dir = params['temp_dir']    
    out_dir = params['out_dir']    

In [107]:
caption_df.loc[(caption_df['raw_caption'] == caption_df['ann_caption']), 'dist_from_raw'] = 0

In [111]:
def compare_two_sentences(sentence_1, sentence_2):
    sentences = [sentence_1, sentence_2]
    
    vectorizer = Vectorizer()
    vectorizer.bert(sentences)
    vec_1, vec_2 = vectorizer.vectors
    
    dist = spatial.distance.cosine(vec_1, vec_2)
    return dist

In [115]:
tqdm.pandas()

In [116]:
caption_df.loc[list(caption_df['dist_from_raw'].isnull().index), 'dist_from_raw'] = \
    caption_df.loc[(caption_df['dist_from_raw'].isnull())].progress_apply(lambda x: compare_two_sentences(x['raw_caption'], x['ann_caption']), axis=1)

100%|██████████| 9/9 [00:13<00:00,  1.48s/it]


In [119]:
caption_df['dist_from_raw'] = caption_df['dist_from_raw'].fillna(0)

In [120]:
caption_df

,raw_caption,level_0,level_1,ann_caption,dist_from_raw
0,a man riding a skateboard up the side of a ramp,31093,2157893,a man riding a skateboard up the side of a ramp,0.000000
1,a man riding a skateboard up the side of a ramp,31093,1233457,a man riding a skateboard up the side of a ramp,0.000000
2,a man riding a skateboard up the side of a ramp,31093,453234,a man riding a skateboard up the side of a ramp,0.000000
3,a man riding a skateboard up the side of a ramp,31093,638282,a man riding a skateboard up the side of a ramp,0.000000
4,a man riding a skateboard up the side of a ramp,31093,1421746,a man riding a skateboard up the side of a ramp,0.000000
5,a man riding a skateboard up the side of a ramp,31093,1475365,a man riding a skateboard up the side of a ramp,0.000000
6,a blue and white bus parked in a parking lot,311909,166445,a car parked on the side of a road,0.025962
7,a cat sitting on the hood of a car,466156,135585,a black and white dog standing next to a black...,0.031496
8,a cat sitting on the hood of a car,466156,243663,a cat sitting on the hood of a car,0.000000
9,a cat sitting on the hood of a car,466156,1358425,a cat sitting on the hood of a car,0.000000


In [121]:
caption_df.loc[caption_df['level_0']==466156]

,raw_caption,level_0,level_1,ann_caption,dist_from_raw
7,a cat sitting on the hood of a car,466156,135585,a black and white dog standing next to a black...,0.031496
8,a cat sitting on the hood of a car,466156,243663,a cat sitting on the hood of a car,0.000000
9,a cat sitting on the hood of a car,466156,1358425,a cat sitting on the hood of a car,0.000000
10,a cat sitting on the hood of a car,466156,47170,a couple of motorcycles parked next to each other,0.076799
11,a cat sitting on the hood of a car,466156,1344825,a cat sitting on the hood of a car,0.000000


In [132]:
pwd

'/home/jdlevy/Documents/UCSD/Senior/Quarter 2/DSC 180B/180b_capstone_xai/src'